주식 시장에는 주식을 팔고자 하는 사람과 사고자 하는 사람, 이렇게 크게 두 유형이 있습니다. 팽팽한 줄다리기 끝에 팔고자 하는 사람이 더 많으면 공급 초과로 가격은 하락하게 됩니다.

주가에 영향을 주는 요소는 금융 애널리스트의 예측, 기관 예측, 경제 위기, 그리고 이를 전달하는 뉴스 매체 등으로 셀 수 없이 많지만 결국 그 종착지에는 대중의 움직임이 있습니다. 그러한 대중의 움직임이 파는 쪽에 더 치우치면 가격은 떨어지고, 사는 쪽에 치우치면 가격은 상승합니다.

그렇다면 우리는 지금 이시간, 대중의 움직임이 어느 방향으로 기울고 있는가를 분석해볼 수 있습니다. 그 대상은 특정 종목이나 특정 국가가 될 수도 있고 주식 시장 자체가 될 수 있습니다. 본 분석은 후자, 주식 시장 자체에 대해 사람들이 어떻게 평가하고 있는가를 분석해볼 것이며 시점은 편의상 '이번 주'로 합니다. 즉, '이번 주 주식 시장에 대한 사람들의 평가는 어떠한가?'에 대한 분석이 되겠습니다. 

>toc
```
Step 1. 블로그 정보 수집
     1-1. 웹사이트 구조 및 데이터 호출 정보 확인
     1-2. 단일 페이지 데이터 호출
     1-3. 다중 페이지 데이터 호출(함수)
Step 2. 블로그 내용 수집
     2-1. HTML 태그 정보 확인
     2-2. 데이터 호출 정보 확인
     2-3. 단일 페이지 데이터 호출(함수)
Step 3. 감성 분류
     3-1. 허깅페이스 모델 검색
     3-2. 추론 모델 인스턴스 생성 및 테스트
     3-3. 단일 포스팅 내 텍스트 감성 분류
     3-4. 전체 7일간 포스팅 감성 분류
     3-5. 결과
```

## Step 1. 블로그 정보 수집

### 1-1. 웹사이트 구조 및 데이터 호출 정보 확인

파이썬으로 포스팅을 긁어올 것이므로 타겟을 명확히 확인합니다. '네이버 블로그'(https://section.blog.naver.com/) 사이트에 진입해서 '주가 전망'이라는 키워드를 검색 후 
개발자 모드를 열어 네트워크 탭을 살펴봅니다.

미리보기를 통해 반환된 문서들을 살펴보면 그 중 SearchList.naver에 우리가 찾는 데이터가 있는 것을 확인할 수 있습니다. 이제 파이썬으로 해당 자료를 가져올 수 있으면 됩니다.

먼저 해당 데이터가 어떤 방식(GET, POST)으로, 어떤 url 값으로 반환받을 수 있는지 살펴본 다음, 하단 쿼리 문자열 매개변수를 확인함으로써 넘겨줄 요청값들을 지정해줄 수 있습니다.

![img](./img/1.png)

### 1-2. 단일 페이지 데이터 호출

currentPage는 현재 페이지 번호, countPerPage는 페이지당 포함하는 포스팅 수, endDate와 startDate는 지난 7일간을 지정해주고 keyword는 검색한 값을 보여줍니다. 이렇게 파라미터 파악이 끝났으면 코드 작성을 위해 필요한 라이브러리를 호출하겠습니다.

In [4]:
import pandas as pd

import requests 
from bs4 import BeautifulSoup

endDate, startDate는 직접 문자열로 입력해줘도 되지만 스크래퍼를 개발할 때에는 항상 자동화를 염두해두어야 합니다. 따라서 오늘 날짜를 기준으로 지난 7일을 계산할 수 있도록 다음과 같이 함수를 사용하겠습니다.

In [5]:
from datetime import datetime, timedelta

datetime.today() - timedelta(30)

datetime.datetime(2022, 9, 20, 0, 4, 58, 668866)

In [6]:
datetime.today().strftime('%Y-%m-%d')

'2022-10-20'

이제, url, header, date, params를 정의하고 파라미터가 잘 세팅되는지 확인합니다.

In [16]:
url = 'https://section.blog.naver.com/ajax/SearchList.naver'
header = {
    'Referer': 'https://section.blog.naver.com/Search/Post.naver',
}

end_date = datetime.today()
start_date = end_date - timedelta(days=7)


params = {
    'countPerPage': 7,
    'currentPage': 1,
    'endDate': end_date.strftime('%Y-%m-%d'),
    'keyword': '주가 전망',
    'orderBy': 'sim',
    'startDate': start_date.strftime('%Y-%m-%d'),
    'type': 'post',
}


In [17]:
params

{'countPerPage': 7,
 'currentPage': 1,
 'endDate': '2022-10-20',
 'keyword': '주가 전망',
 'orderBy': 'sim',
 'startDate': '2022-10-13',
 'type': 'post'}

requests 모듈을 통해 데이터를 불러옵니다. 결과값을 확인했을 때 우리가 앞서 봤던 SearchList.naver의 미리보기 값과 형태가 동일해야 합니다. 텍스트 앞부분만 짧게 출력하겠습니다.

In [18]:
res = requests.get(url, params=params, headers=header)

In [20]:
res.text[:1000]

')]}\',\n{"result":{"searchDisplayInfo":{"authUrlType":"LOGIN","authUrl":"https://nid.naver.com/nidlogin.login?svctype=128&a_version=2&viewtype=1&&url=https://section.blog.naver.com/Search/Post.naver?keyword=%25EC%25A3%25BC%25EA%25B0%2580%2B%25EC%25A0%2584%25EB%25A7%259D&surl=https://section.blog.naver.com","existSuicideWord":false,"keyword":"주가 전망","eucKrEncodedKeyword":"%C1%D6%B0%A1+%C0%FC%B8%C1","displayType":"NORMAL"},"searchList":[{"domainIdOrBlogId":"wave7906","logNo":222903580961,"gdid":"90000003_0000000000000033E616B921","postUrl":"https://blog.naver.com/wave7906/222903580961","title":"한국가스공사 <strong class=\\"search_keyword\\">주가 전망</strong> 겨울관련주,테마주 분석 정리","noTagTitle":"한국가스공사 주가 전망 겨울관련주,테마주 분석 정리","contents":"오늘은 한국가스공사 <strong class=\\"search_keyword\\">주가 전망</strong>에 대해 살펴보겠습니다. 이제 가을이고 조금 있으면 금방 겨울이 됩니다. 그렇다면 우리는 겨울에 관심을 많이 받는 가스공사 주가를 생각 안 할 수가 없겠죠. 그래서 오늘은 현재 분위기는 어떻게 전문가들은 어떻게 생각하고 있는지 한번 알아보도록 하겠습니다.... ","nickName":"cool","blogName":"LIFE PLUS","profileImgUrl":"htt

데이터를 json 형태로 변환해주면 좋겠습니다. 그 전에 앞부분의 불필요한 문자열을 잘라줘야 정상적으로 변환이 될 것입니다.

In [23]:
import json

json.loads(res.text.split('\n')[1])['result']['searchList']

[{'domainIdOrBlogId': 'wave7906',
  'logNo': 222903580961,
  'gdid': '90000003_0000000000000033E616B921',
  'postUrl': 'https://blog.naver.com/wave7906/222903580961',
  'title': '한국가스공사 <strong class="search_keyword">주가 전망</strong> 겨울관련주,테마주 분석 정리',
  'noTagTitle': '한국가스공사 주가 전망 겨울관련주,테마주 분석 정리',
  'contents': '오늘은 한국가스공사 <strong class="search_keyword">주가 전망</strong>에 대해 살펴보겠습니다. 이제 가을이고 조금 있으면 금방 겨울이 됩니다. 그렇다면 우리는 겨울에 관심을 많이 받는 가스공사 주가를 생각 안 할 수가 없겠죠. 그래서 오늘은 현재 분위기는 어떻게 전문가들은 어떻게 생각하고 있는지 한번 알아보도록 하겠습니다.... ',
  'nickName': 'cool',
  'blogName': 'LIFE PLUS',
  'profileImgUrl': 'https://blogpfthumb-phinf.pstatic.net/MjAxODA0MDNfMTc4/MDAxNTIyNzM3MTg0MDY4.PNHBWFmo3WqRj_VxJhYBquLwEdcVV7bNGXIybX2SO8Qg.scFxmzGSB73616-r3E8OErb-yUVgdWTWe5pR-w_3EPsg.JPEG.wave7906/20171027_074509.jpg?type=s1',
  'addDate': 1666126800000,
  'thumbnails': [{'url': 'https://blogthumb.pstatic.net/MjAyMjEwMThfMjI0/MDAxNjY2MDcxNjg4MzY5.YZabCmsVhBMUZgBYZfFpykm_4AjcuSx9QUNjQlJAsvAg.YuVBKFYNZSC2RpO1AY_X6DUYuunCEWbKAIp_

### 1-3. 다중 페이지 데이터 호출(함수)

이제, 앞서 구현한 코드들을 종합하여 키워드와 페이지 번호만 넣어주면 지난 7일간의 포스팅 정보를 긁어올 수 있는 함수를 정의하겠습니다.

In [26]:
def get_posts_info(keyword, start_page=1, end_page=10):
    url = 'https://section.blog.naver.com/ajax/SearchList.naver'
    header = {
        'Referer': 'https://section.blog.naver.com/Search/Post.naver',
    }

    end_date = datetime.today()
    start_date = end_date - timedelta(days=7)

    all_posts_info = []
    print(f'[INFO] keyword : {keyword}, start_page : {start_page}, end_page : {end_page}')
    for i in range(start_page, end_page+1):
        params = {
            'countPerPage': 7,
            'currentPage': i,
            'endDate': end_date.strftime('%Y-%m-%d'),
            'keyword': keyword,
            'orderBy': 'sim',
            'startDate': start_date.strftime('%Y-%m-%d'),
            'type': 'post',
        }

        res = requests.get(url, params=params, headers=header)
        current_posts_info = json.loads(res.text.split('\n')[1])['result']['searchList']
        all_posts_info += current_posts_info
        print(f'[INFO] 포스트 정보 수집 중 .. (page : {i}/{end_page} current posts : {len(current_posts_info)} all posts : {len(all_posts_info)})')
    return all_posts_info

In [36]:
posts_info = get_posts_info('주가전망', 1, 10)

[INFO] keyword : 주가전망, start_page : 1, end_page : 10
[INFO] 포스트 정보 수집 중 .. (page : 1/10 current posts : 7 all posts : 7)
[INFO] 포스트 정보 수집 중 .. (page : 2/10 current posts : 7 all posts : 14)
[INFO] 포스트 정보 수집 중 .. (page : 3/10 current posts : 7 all posts : 21)
[INFO] 포스트 정보 수집 중 .. (page : 4/10 current posts : 7 all posts : 28)
[INFO] 포스트 정보 수집 중 .. (page : 5/10 current posts : 7 all posts : 35)
[INFO] 포스트 정보 수집 중 .. (page : 6/10 current posts : 7 all posts : 42)
[INFO] 포스트 정보 수집 중 .. (page : 7/10 current posts : 7 all posts : 49)
[INFO] 포스트 정보 수집 중 .. (page : 8/10 current posts : 7 all posts : 56)
[INFO] 포스트 정보 수집 중 .. (page : 9/10 current posts : 7 all posts : 63)
[INFO] 포스트 정보 수집 중 .. (page : 10/10 current posts : 7 all posts : 70)


In [37]:
len(posts_info)

70

총 10개 페이지, 70개 포스팅 정보가 수집되었습니다.

## Step 2. 블로그 내용 수집

### 2-1. HTML 태그 정보 확인

이렇게 수집한 포스팅 정보로 각 url을 순회하며 텍스트만 뽑아올 것입니다. 수집한 url중 하나에 진입해서 html 태그를 살펴봅니다.

![img](./img/2.png)

본문 텍스트는 `se-main-continer`라는 div class가 지정되어 있습니다. 

### 2-2. 데이터 호출 정보 확인

데이터 호출을 위해 이번에는 네트워크탭의 '문서'를 살펴보면 PostView.naver에 우리가 찾는 본문 텍스트가 있습니다. 해당 데이터를 어떻게 호출할 수 있을지 url과 매개변수를 통해 살펴봅니다.

![img](./img/3.png)

### 2-3. 단일 페이지 데이터 호출(함수)

이번에는 함수로 바로 만들어주겠습니다.

select_one 함수로 본문을 가져오고 select('p')를 해줌으로써 본문 내 모든 문장들을 리스트로 추출합니다.  
또한, list comprehension 문법을 통해 각 리스트 내 요소에서 텍스트만 추출해 다시 리스트로 저장하고, 제어문자(u200b는 폭 없는 공백입니다. 네이버 블로그 포스팅시 자동으로  문자 사이에 포함됩니다.)는 없애줍니다.  
계속해서 list comprehension 문법을 사용하며 줄바꿈은 분리해주고 공백 요소는 제거해 깔끔한 형태의 문자열만 담은 리스트로 만듭니다.


In [49]:
def get_posts(x):
    url = 'https://blog.naver.com/PostView.naver'
    params = {
        'blogId': x['domainIdOrBlogId'],
        'logNo': x['logNo'],
        'redirect': 'Dlog',
        'widgetTypeCall': 'true',
        'directAccess': 'false'
    }

    res = requests.get(url, params=params)
    soup = BeautifulSoup(res.text, 'lxml')
    posts = soup.select_one('.se-main-container').select('p')
    posts = [x.get_text().replace('\u200b','') for x in posts]
    
    filtered_posts = [x.replace('다. ', '다. \n') for x in posts]
    filtered_posts = sum([x.split('\n') for x in posts],[])
    filtered_posts = [x.strip() for x in filtered_posts if not x in ['',' ']]
    return filtered_posts

In [51]:
get_posts(posts_info[0])

['안녕하세요. 오늘은 한국가스공사 주가 전망에 대해 살펴보겠습니다. 이제 가을이고 조금 있으면 금방 겨울이 됩니다. 그렇다면 우리는 겨울에 관심을 많이 받는 가스공사 주가를 생각 안 할 수가 없겠죠. 그래서 오늘은 현재 분위기는 어떻게 전문가들은 어떻게 생각하고 있는지 한번 알아보도록 하겠습니다.',
 '주식 종목은 각각 특성이 존재하고 계절에 따라 수익은 달라집니다. 저도 이번에 알았네요. 주식에는 다양한 테마주가 존재합니다. 오늘은 계절주로서 겨울과 관련 있어요. 겨울에는 난방, 가스, 보일러에 대한 계절주가 상승하게 됩니다. 그럼 여름은 어떨까요. 더운 날씨를 시원하게 식혀줄 피서철 여행주를 보면 복날과 관련한 하림, 마니커 등의 주가도 함께 상승하죠. 해당 계절마다 상승하기 때문에 미리 매수를 해둔다면 좋을 거 같아요. 이런 시기에 맞춰 종목을 정해서 매수를 한다면 계획에 맞춰 좋은 결과를 가져올 수 있겠죠.',
 '▣그럼 한국가스공사를 살펴볼까요.',
 '한국가스공사는 1983년도에 설립되어 99년에 성장을 한 기업입니다. 주로 천연가스판매를 주력으로 하고 있으며 해외 천연가스 생산지에서 LNG를 수입해 국내 발전사나 도시가스 회사에 공급을 하는 역할을 합니다.',
 '기후변화와 전기자동차처럼 친환경에너지 수요가 증가하게 되면서 현재는 친환경 에너지, 수소발전으로 전환하기 위해 여기에 필요한 개발과 인프라구축을 진행하고 있다고 합니다.',
 '실제로 국내 유일 천연가스 도매사업자인 한국가스공사의 주가는 크게 상승했습니다. 그러나 하반기 주가 방향을 두고 시각은 일관되지 않고 있어요. 미래는 누구도 예측할 수 없으니까요.',
 '천연가스 가격이 상승한 이유는 러시아가 유럽으로 수출하고 있는 천연가스의 양을 크게 줄였기 때문입니다. 그래서 유럽이 대체 수입처를 찾기 시작했죠. 그게 영향을 받은 겁니다.',
 '증권가에선 앞으로 한국가스공사의 주가 흐름을 보면서 분석이 엇갈립니다. 오름세를 보일 거라는 의견도 있는데 원인으로는 겨울철에 사용량이 증가하는 

In [52]:
get_posts(posts_info[1])

['반도체 관련주 하면 가장 먼저 떠오르는 주식은 엔비디아입니다. 최근 매파적인 금리 인상과 다양한 수요 감소 등의 악재로 인해 엔비디아 주가는 고점 대비 약 -68% 정도 하락을 한 상태입니다.',
 'NVIDIA 엔비디아 미국 주식의 주가와 전망 실적에 대해 알아보겠습니다.',
 'NVDA',
 '대만 출신인 젠슨 황이 1993년 그래픽 카드 설계를 위해 설립한 미국 회사로 반도체를 설계하는 팹리스 회사입니다. 컴퓨터 CPU 설계로 GPU 리테일 시장에서 압도적인 점유율을 차지하고 있습니다.',
 'Graphics와 Compute & Networking 두 부분을 통해 운영되고 있으며 크게 게이밍, 전문 시각화, 데이터 센터, 자동차 시장을 대상으로 매출을 만들어 내고 있습니다.',
 'Graphics 부문은 GPU, GeForce NOW 게임 스트리밍 서비스 및 관련, RTX GPU, 기반 비주얼, 가상 컴퓨팅을 위한 NVIDIA vGPU 소프트웨어, 인포테인먼트 시스템용 자동차 플랫폼 등이 포함됩니다.',
 'Compute & Networking 부문에는 인공 지능(AI), 고성능 컴퓨팅(HPC) 및 Mellanox 네트워킹 상호 연결 솔루션, 자율 주행 개발 계약 & 차량 솔루션, 컴퓨팅을 위한 데이터 센터 플랫폼 등의 시스템이 포함됩니다.',
 '엔비디아 주가',
 '엔비디아는 2021년 11월 22일 346.47로 최고가를 달성한 뒤 쭉 흘러내려서 현재 NVIDIA 주가는 2022년 10월 18일 장 마감 기준 119.67달러입니다.',
 '성장률 둔화, 글로벌 경기 침체 우려, 암호화폐 시장 침체, 중국 반도체 수출 규제, PC 수요 감소 등 다양한 악재들이 겹치면서 하락하였습니다.',
 '특히 암호화폐를 채굴하기 위해서는 그래픽 카드가 필요하였고 이에 따른 수요가 크게 증가하였습니다. 그러나 암호화폐 침체와 더불어 이더리움의 채굴 방식 변경으로 수요가 확 줄어들면서 NVDA 주가는 크게 하락하였습니다.',
 '현재 시가 총액은 425조 2150

## Step 3. 감성 분류

이제, 이렇게 얻은 문자열에 대해 감성 분류를 실시할 수 있습니다. 텍스트 클랜징 작업은 최대한 정교하게 할 수록 좋습니다. 텍스트 수집에 정도가 없듯 텍스트 전처리 방법에도 역시 정도가 없습니다.

이번 튜토리얼에서는 간단하게 전처리한 문장을 활용해 감성 분류를 해봅니다.

### 3-1. 허깅페이스 모델 검색

우선, 임의의 텍스트 한 줄을 가져다 잘 분류해줄 수 있는 모델을 찾아봅니다. 

언어모델의 경우 대규모로 사전학습된 모델을 먼저 리서치해보는 것이 좋습니다. 간단한 분석을 위해 몇 주간의 학습 시간을 소진하기에는 현실적으로 어렵기 때문에, 언어모델 사용시 다운스트림 테스크로 우리의 데이터셋에 조금 더 확률값을 높여주고 우리가 목표로하는 결과값을 출력해내도록 구조를 맞춰주는 정도의 파인튜닝이 일반적입니다.

감성 분류, 그 중에서도 금융 텍스트에 특화된 한국어 모델이 이미 허깅페이스에 올라와있습니다. 해당 모델을 테스트해보고 사용 여부를 판단합니다.

![img](./img/4.png)

부정적 어감의 문장을 가져다 넣으니 negative로 잘 분류해줍니다. 중립은 어떨까요?

![img](./img/5.png)

'엔비디아 전망'이라는 텍스트는 97.6%의 확률로 neutral으로 분류를 해줍니다. 만약 우리가 사용하는 모델이 긍/부정으로만 분류할 수 있다면 중립에 해당하는 문장은 최대한 걸러내야 할 것입니다.

여기서는 우리가 수집한 전체 문장에 대해 중립, 부정, 긍정 각각의 비중을 살펴보고 주식시장의 동향을 파악해보고자 합니다. 

참고로 우리는 특정 블로그 포스팅 자체에 대해 '긍정 포스팅이다.', '부정 포스팅이다.' 하고 분류하기보다 위 방식처럼 하나의 글 내에서도 여러 문장으로 분리하고 각각을 분류 대상으로 삼을 것입니다. 한 명이 작성한 하나의 블로그 포스팅 안에서도 긍정적인 문장과 부정적인 문장이 혼재해있습니다. 해당 블로그 포스팅으로 누군가는 부정적인 정보를 획득해갈 것이고 누군가는 반대로 긍정적인 정보를 획득해갈 것입니다. 사람마다 받아들이는 정보는 매우 주관적이므로 블로그 포스팅 내 모든 문장을 정보 혹은 대중의 판단 근거라 가정하는 것입니다.

물론 이러한 분석 방식 역시 제 주관과 의도가 강하게 개입되어 있으며, 반드시 정답은 아닙니다.



### 3-2. 추론 모델 인스턴스 생성 및 테스트

In [55]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-FinBert-SC')
model = AutoModelForSequenceClassification.from_pretrained('snunlp/KR-FinBert-SC')

senti_classifier = pipeline(task='text-classification', model=model, tokenizer=tokenizer)

위 코드는 허깅페이스를 통해 우리가 사용하려는 모델과 토크나이저를 불러와 빠르게 추론 모델을 만듭니다.  
해당 모델은 뉴스기사 텍스트로 1차 파인튜닝된 모델입니다. 그렇기 때문에 추가 학습 없이 우리 테스크에 바로 적용할 수 있습니다. 

다만, 경우에 따라 같은 유형의 테스크라 하더라도 모델이 우리 데이터에 완전히 맞지는 않을 수 있습니다. 따라서 몇가지 단어를 넣어 적절한 출력이 나오는지 한번 더 확인하겠습니다.

In [59]:
senti_classifier(['상승','하락','애플'])

[{'label': 'positive', 'score': 0.8705503940582275},
 {'label': 'negative', 'score': 0.9346325993537903},
 {'label': 'neutral', 'score': 0.5876992344856262}]

주가 관련 텍스트를 학습한 탓에 상승이라는 키워드에는 positive를, 하락이라는 키워드에는 negative를, 애플과 같은 고유명사에는 neutral을 출력해줍니다. 우리가 기대했던 결과값입니다.

그럼, 이제 지난 7일간의 주식 시장에 대한 평가를 네이버 블로그에 한정해서 살펴보도록 하겠습니다.

### 3-3. 단일 포스팅 내 텍스트 감성 분류

In [62]:
len(posts_info)

70

우리가 분석하려는 총 포스트 수는 총 70개입니다. 첫번째 포스팅을 먼저 분석해보겠습니다.

In [64]:
get_posts(posts_info[0])

['안녕하세요. 오늘은 한국가스공사 주가 전망에 대해 살펴보겠습니다. 이제 가을이고 조금 있으면 금방 겨울이 됩니다. 그렇다면 우리는 겨울에 관심을 많이 받는 가스공사 주가를 생각 안 할 수가 없겠죠. 그래서 오늘은 현재 분위기는 어떻게 전문가들은 어떻게 생각하고 있는지 한번 알아보도록 하겠습니다.',
 '주식 종목은 각각 특성이 존재하고 계절에 따라 수익은 달라집니다. 저도 이번에 알았네요. 주식에는 다양한 테마주가 존재합니다. 오늘은 계절주로서 겨울과 관련 있어요. 겨울에는 난방, 가스, 보일러에 대한 계절주가 상승하게 됩니다. 그럼 여름은 어떨까요. 더운 날씨를 시원하게 식혀줄 피서철 여행주를 보면 복날과 관련한 하림, 마니커 등의 주가도 함께 상승하죠. 해당 계절마다 상승하기 때문에 미리 매수를 해둔다면 좋을 거 같아요. 이런 시기에 맞춰 종목을 정해서 매수를 한다면 계획에 맞춰 좋은 결과를 가져올 수 있겠죠.',
 '▣그럼 한국가스공사를 살펴볼까요.',
 '한국가스공사는 1983년도에 설립되어 99년에 성장을 한 기업입니다. 주로 천연가스판매를 주력으로 하고 있으며 해외 천연가스 생산지에서 LNG를 수입해 국내 발전사나 도시가스 회사에 공급을 하는 역할을 합니다.',
 '기후변화와 전기자동차처럼 친환경에너지 수요가 증가하게 되면서 현재는 친환경 에너지, 수소발전으로 전환하기 위해 여기에 필요한 개발과 인프라구축을 진행하고 있다고 합니다.',
 '실제로 국내 유일 천연가스 도매사업자인 한국가스공사의 주가는 크게 상승했습니다. 그러나 하반기 주가 방향을 두고 시각은 일관되지 않고 있어요. 미래는 누구도 예측할 수 없으니까요.',
 '천연가스 가격이 상승한 이유는 러시아가 유럽으로 수출하고 있는 천연가스의 양을 크게 줄였기 때문입니다. 그래서 유럽이 대체 수입처를 찾기 시작했죠. 그게 영향을 받은 겁니다.',
 '증권가에선 앞으로 한국가스공사의 주가 흐름을 보면서 분석이 엇갈립니다. 오름세를 보일 거라는 의견도 있는데 원인으로는 겨울철에 사용량이 증가하는 

In [84]:
x = get_posts(posts_info[0])
y = [x['label'] for x in senti_classifier(x)]

df = pd.DataFrame(data={
    'text': x,
    'senti': y
})

df

,text,senti
0,안녕하세요. 오늘은 한국가스공사 주가 전망에 대해 살펴보겠습니다. 이제 가을이고 조...,neutral
1,주식 종목은 각각 특성이 존재하고 계절에 따라 수익은 달라집니다. 저도 이번에 알았...,neutral
2,▣그럼 한국가스공사를 살펴볼까요.,neutral
3,한국가스공사는 1983년도에 설립되어 99년에 성장을 한 기업입니다. 주로 천연가스...,neutral
4,"기후변화와 전기자동차처럼 친환경에너지 수요가 증가하게 되면서 현재는 친환경 에너지,...",positive
5,실제로 국내 유일 천연가스 도매사업자인 한국가스공사의 주가는 크게 상승했습니다. 그...,positive
6,천연가스 가격이 상승한 이유는 러시아가 유럽으로 수출하고 있는 천연가스의 양을 크게...,neutral
7,증권가에선 앞으로 한국가스공사의 주가 흐름을 보면서 분석이 엇갈립니다. 오름세를 보...,negative
8,이미 반면 원료 가격 상승분을 판매가인 가스 요금에 전가하지 못한다는 점이 리스크로...,negative


### 3-4. 전체 7일간 포스팅 감성 분류

서론, 혹은 이야기를 풀어나가기 위한 담화용 텍스트는 neutral로 분류되고 그 외 주가 판단에 대한 내용은 positive 혹은 negative로 분류되는 모습을 볼 수 있습니다.

이번에는 전체 70개 포스팅에 대해 모두 분류하고 분류 비중을 살펴보도록 하겠습니다.

포스팅을 먼저 모두 수집하고 전체 데이터에 대해 한번에 추론하는 방식 혹은 매 포스팅마다 수집과 추론을 반복해서 결과값을 합치는 방식을 취할 수 있습니다.

속도 및 작업 편의를 고려하면 전자가 좋겠지만 각 포스팅에 수십 개의 문장이 있기 때문에 전체 분류 타겟이 메모리에 한번에 올라가게 되면 중간에 멈출 수도 있습니다. 따라서 배치 작업으로 안정적인 추론이 이루어질 수 있도록 후자 형태로 코드를 구성하겠습니다.

In [85]:
for i in range(1, len(posts_info)): # 첫번째(index:0) 포스팅에 이어서 두번째(index:1) 포스팅부터 끝까지 수집합니다.
    x = get_posts(posts_info[i])
    y = [x['label'] for x in senti_classifier(x)]

    df_next = pd.DataFrame(data={
        'text': x,
        'senti': y
    })

    df = pd.concat([df, df_next])
    print(f'[INFO] 분류 작업 중.. (타겟 포스팅: {i}/{len(posts_info)-1} 분류된 문장 수: {len(df)})')

[INFO] 분류 작업 중.. (타겟 포스팅: 1/69 분류된 문장 수: 35)
[INFO] 분류 작업 중.. (타겟 포스팅: 2/69 분류된 문장 수: 59)
[INFO] 분류 작업 중.. (타겟 포스팅: 3/69 분류된 문장 수: 85)
[INFO] 분류 작업 중.. (타겟 포스팅: 4/69 분류된 문장 수: 244)
[INFO] 분류 작업 중.. (타겟 포스팅: 5/69 분류된 문장 수: 363)
[INFO] 분류 작업 중.. (타겟 포스팅: 6/69 분류된 문장 수: 394)
[INFO] 분류 작업 중.. (타겟 포스팅: 7/69 분류된 문장 수: 410)
[INFO] 분류 작업 중.. (타겟 포스팅: 8/69 분류된 문장 수: 423)
[INFO] 분류 작업 중.. (타겟 포스팅: 9/69 분류된 문장 수: 520)
[INFO] 분류 작업 중.. (타겟 포스팅: 10/69 분류된 문장 수: 563)
[INFO] 분류 작업 중.. (타겟 포스팅: 11/69 분류된 문장 수: 581)
[INFO] 분류 작업 중.. (타겟 포스팅: 12/69 분류된 문장 수: 692)
[INFO] 분류 작업 중.. (타겟 포스팅: 13/69 분류된 문장 수: 728)
[INFO] 분류 작업 중.. (타겟 포스팅: 14/69 분류된 문장 수: 799)
[INFO] 분류 작업 중.. (타겟 포스팅: 15/69 분류된 문장 수: 892)
[INFO] 분류 작업 중.. (타겟 포스팅: 16/69 분류된 문장 수: 906)
[INFO] 분류 작업 중.. (타겟 포스팅: 17/69 분류된 문장 수: 1084)
[INFO] 분류 작업 중.. (타겟 포스팅: 18/69 분류된 문장 수: 1146)
[INFO] 분류 작업 중.. (타겟 포스팅: 19/69 분류된 문장 수: 1169)
[INFO] 분류 작업 중.. (타겟 포스팅: 20/69 분류된 문장 수: 1211)
[INFO] 분류 작업 중.. (타겟 포스팅: 21/69 분류된 문장 수: 1230)
[INFO] 분류 작업 중.. (타겟

In [91]:
df = df.reset_index(drop=True)
df

,text,senti
0,안녕하세요. 오늘은 한국가스공사 주가 전망에 대해 살펴보겠습니다. 이제 가을이고 조...,neutral
1,주식 종목은 각각 특성이 존재하고 계절에 따라 수익은 달라집니다. 저도 이번에 알았...,neutral
2,▣그럼 한국가스공사를 살펴볼까요.,neutral
3,한국가스공사는 1983년도에 설립되어 99년에 성장을 한 기업입니다. 주로 천연가스...,neutral
4,"기후변화와 전기자동차처럼 친환경에너지 수요가 증가하게 되면서 현재는 친환경 에너지,...",positive
...,...,...
4359,현재 미중갈등이 심화되며 중국을 아예 배재하고 있기 때문에 중국이 잡고있는 수요에대...,neutral
4360,쉽게 끝날 단기적 이슈는 아니라고 보여집니다.,neutral
4361,해당종목과 함께 이브이첨단소재도 같은테마로 비슷한 흐름을 보여주고 있기때문에 쭉 지...,positive
4362,STX주가전망에 대하여 짧게나마 알아보았습니다.,neutral


### 3-5. 결과

In [93]:
df['senti'].value_counts() / len(df)

neutral     0.725710
positive    0.188130
negative    0.086159
Name: senti, dtype: float64

결과는 중립 73%, 긍정 18%, 부정 8%로 현재 불안정한 시장 상황을 충분히 반영하고 있습니다.  
사실, 이는 해석하기 나름입니다. '여전히 긍정적인 전망을 하는 사람들이 더 많기 때문에 아직 바닥이 아니다'라고 할 수도 있고, '이제 바닥을 찍고 상승하려는 움직임이 보인다'라고 할 수도 있습니다.  
따라서 이렇게 하나의 지표나 결과를 보고 전체 주식시장을 판단하는 것은 무리가 있겠습니다만, 이러한 비율을 매주 추적하며 시계열로 분석한다면 더 유의미한 추세적 지표로 활용할 수 있을 것입니다.